In [1]:
import numpy as np
import numpy.linalg as lyn
import random
import urllib.request as urlreq
import re
import matplotlib.pyplot as plt
%matplotlib inline
%autosave 180
import Senator as sen
from itertools import combinations

Autosaving every 180 seconds


# Consider a simpler version of the SVD on a covariance (symmetric) matrix:

In [5]:
def simpleSVD(A):
    #Make AA^T matrix
    A2 = np.matmul(A,A.T)
    #Get the eigens
    l , e = lyn.eig(A2)
    #Same for A^TA -> For covariance this is the same as AA^T
    A3 = np.matmul(A.T,A)
    l2,e2 = lyn.eig(A3)
    #Get the singular Values and order them
    l=np.sqrt(l)
    s=l[np.flip(np.argsort(l))]
    #Reorder the eigenvector columns to match order of singular values
    e=e[:,np.flip(np.argsort(l))]
    #Same for other AA matrix
    l2=np.sqrt(l2)
    s2=l2[np.flip(np.argsort(l2))]
    e2=e2[:,np.flip(np.argsort(l2))]
    #Return the ordered UsV matrices
    return e, s, e2.T

In [11]:
#Sample covariance matrix (symmetric)
A=np.array([[1,2,3],[2,1,4],[3,4,1]])
print(A)
U,s,V = simpleSVD(A)
print("U=",U)
print("s=", s)
print("V=", V)

[[1 2 3]
 [2 1 4]
 [3 4 1]]
U= [[ 0.50578521 -0.25523155  0.82403773]
 [ 0.58437383 -0.60130182 -0.54492509]
 [ 0.63457746  0.75716113 -0.15497893]]
s= [7.07467358 3.1878826  0.88679099]
V= [[ 0.50578521  0.58437383  0.63457746]
 [-0.25523155 -0.60130182  0.75716113]
 [ 0.82403773 -0.54492509 -0.15497893]]


In [12]:
#Use Numpy
Ut,st,Vt=np.linalg.svd(A, full_matrices=False)
print("UTrue=",Ut)
print("sTrue=", st)
print("VTrue=", Vt)

UTrue= [[-0.50578521 -0.25523155 -0.82403773]
 [-0.58437383 -0.60130182  0.54492509]
 [-0.63457746  0.75716113  0.15497893]]
sTrue= [7.07467358 3.1878826  0.88679099]
VTrue= [[-0.50578521 -0.58437383 -0.63457746]
 [ 0.25523155  0.60130182 -0.75716113]
 [ 0.82403773 -0.54492509 -0.15497893]]


In [13]:
#Multiply my computed values
np.dot(U, np.dot(np.diag(s), V))

array([[2.61966726, 2.18208912, 1.54137294],
       [2.18208912, 3.83189977, 1.24700891],
       [1.54137294, 1.24700891, 4.69778014]])

In [14]:
#Multiply Numpys values
np.dot(Ut, np.dot(np.diag(st), Vt))

array([[1., 2., 3.],
       [2., 1., 4.],
       [3., 4., 1.]])

Looking at my output vs the svd call output, I have the same matrices, but the signs of some columns(rows) are wrong in terms of recombining to get the original matrix. Below, I attempt to fix it with a permutation comparison, but its not going well..

# Trying to fix the sign error (Probably bad)

In [ ]:
def mySvd(A):
    m,n = A.shape
    A2 = np.matmul(A,A.T)
    l,e = lyn.eig(A2)
    l=np.sqrt(l)
    s=l[np.flip(np.argsort(l))]
    e=e[:,np.flip(np.argsort(l))]
    A3 = np.matmul(A.T,A)
    l2,e2 = lyn.eig(A3)
    l2=np.sqrt(l2)
    s2=l2[np.flip(np.argsort(l2))]
    e2=e2[:,np.flip(np.argsort(l2))]
    badCols=[]
    badS=[]
    for i in range(0, A2.shape[0]):
        if(s[i]==0):
            badS.append(i)
        for j in range(0,A2.shape[1]):
            if(e[i,j]==1):
                badCols.append(i)
    e=np.delete(e, badCols, 1)
    e=np.delete(e, badS,1)
    s=np.delete(s,badS,0)
    udim = e.shape[1]
    vdim = e2.shape[1]
    uperms, vperms = getPerms(udim,vdim)
    errmin=10
    uOpt=1
    vOpt=1
    print(e)
    print(s)
    print(e2.T)
    for us in uperms:
        for vs in vperms:
            testA=np.dot(np.matmul(e,us), np.dot(np.diag(s), np.matmul(vs,e2.T)))
            err = np.linalg.norm(A-testA)
            print(err)
            print(us)
            print(vs)
            print('*'*20)
            if(err < errmin):
                errmin=err
                uOpt=us
                vOpt=vs
    e=np.matmul(e,uOpt)
    e2=np.matmul(vOpt,e2.T)
    return e,s,e2.T

In [ ]:
A=np.array([[1,2,3],[2,1,4],[3,4,1]])
print(A)
U,s,V = mySvd(A)
print(U)
print(s)
print(V)

In [ ]:
np.dot(U, np.dot(np.diag(s), V))

In [ ]:
Ut,st,Vt=np.linalg.svd(A, full_matrices=False)
print(Ut)
print(st)
print(Vt)

In [ ]:
for us in u:
    for vs in v:
        print(us,vs)

In [ ]:
def getPerms(udim, vdim):
    uList=[]
    vList=[]
    Pu=[]
    Pv=[]
    P=np.eye(udim)
    for i in range(0,udim+1):
        x=combinations(np.arange(0,udim),i)
        Pu.append(x)
    for combs in Pu:
        for l in combs:
            for i in l:
                P[i,i]=-P[i,i]
            uList.append(P)
            P=np.eye(udim)
    P=np.eye(vdim)
    for i in range(0,vdim+1):
        x=combinations(np.arange(0,vdim),i)
        Pv.append(x)
    for combs in Pv:
        for l in combs:
            for i in l:
                P[i,i]=-P[i,i]
            vList.append(P)
            P=np.eye(vdim)
    return uList, vList

In [ ]:
P=np.eye(3)
P1=[]
mylist=[]
for i in range(0,4):
    x=combinations(np.arange(0,3),i)
    P1.append(x)
for combs in P1:
    for l in combs:
        for i in l:
            P[i,i]=-P[i,i]
        print(P)
        mylist.append(P)
        P=np.eye(3)
print(mylist)

In [ ]:
x=np.arange(0,3)
print(x)

In [ ]:
A=np.array([[-1.55431223e-15, -4.44089210e-16, -8.88178420e-16],
 [ 0.00000000e+00, -6.66133815e-16,  4.44089210e-16],
 [-8.88178420e-16,  4.44089210e-16,  1.22124533e-15]])
np.linalg.norm(A)

In [ ]:
print(i)
            P[i,i]=-P[i,i]
        print(P)
        P=np.eye(3)

In [107]:
def newSVD(A):
    if(np.array_equal(A,A.T)):
        print("Symmetric")
        D,V = np.linalg.eig(np.dot(A.T,A))
        D=np.sqrt(D)
        s=D[np.flip(np.argsort(D))]
        #Reorder the eigenvector columns to match order of singular values
        V=V[:,np.flip(np.argsort(D))]
        U = A.dot(V)/D
        return U,s,V
    else:
        D,V = np.linalg.eig(np.dot(A.T,A))
        D=np.sqrt(D)
        s=D[np.flip(np.argsort(D))]
        #Reorder the eigenvector columns to match order of singular values
        V=V[:,np.flip(np.argsort(D))]
        U = A.dot(V)/D
        return U,D,V

In [108]:
A=np.array([[1,2,3],[2,1,4],[3,4,1]])
print(A)
U,s,V = newSVD(A)
print(U)
print(s)
print(V)
np.dot(U,np.dot(np.diag(s),V.T))

[[1 2 3]
 [2 1 4]
 [3 4 1]]
Symmetric
[[ 0.50578521  0.91751971 -0.22922715]
 [ 0.58437383  2.16159122  0.15158483]
 [ 0.63457746 -2.72188242  0.04311135]]
[7.07467358 3.1878826  0.88679099]
[[ 0.50578521 -0.25523155  0.82403773]
 [ 0.58437383 -0.60130182 -0.54492509]
 [ 0.63457746  0.75716113 -0.15497893]]


array([[ 0.89578781,  0.44304025,  4.5168448 ],
       [ 0.44304025, -1.80081129,  7.82019246],
       [ 4.5168448 ,  7.82019246, -3.72695351]])

In [109]:
A=np.array([[1,2,3],[2,1,4],[3,4,1]])
print(A)
Ut,st,Vt=np.linalg.svd(A, full_matrices=False)
print(Ut)
print(st)
print(Vt)
np.dot(Ut,np.dot(np.diag(st),Vt))

[[1 2 3]
 [2 1 4]
 [3 4 1]]
[[-0.50578521 -0.25523155 -0.82403773]
 [-0.58437383 -0.60130182  0.54492509]
 [-0.63457746  0.75716113  0.15497893]]
[7.07467358 3.1878826  0.88679099]
[[-0.50578521 -0.58437383 -0.63457746]
 [ 0.25523155  0.60130182 -0.75716113]
 [ 0.82403773 -0.54492509 -0.15497893]]


array([[1., 2., 3.],
       [2., 1., 4.],
       [3., 4., 1.]])

array([[ 1.15020617,  2.24412282, -2.18630315],
       [ 2.24412282,  5.03694734, -5.50623164],
       [-2.18630315, -5.50623164,  7.81319102]])

In [111]:
A=np.array([[1,2,3],[2,1,4],[3,4,1],[1,1,1]],dtype=float)
print(A)
U,s,V = newSVD(A)
print(U)
print(s)
print(V)
np.dot(U,np.dot(np.diag(s),V.T))

[[1. 2. 3.]
 [2. 1. 4.]
 [3. 4. 1.]
 [1. 1. 1.]]
[[ 0.49051953  0.926174   -0.23339951]
 [ 0.56694707  2.15973522  0.14695212]
 [ 0.61790503 -2.67717277  0.03638706]
 [ 0.23697042 -0.10348751  0.03666784]]
[7.28198365 0.89493871 3.18932577]
[[ 0.51017681 -0.25421279  0.82164194]
 [ 0.58453532 -0.59828833 -0.54805979]
 [ 0.63090259  0.75988613 -0.15663648]]


array([[1., 2., 3.],
       [2., 1., 4.],
       [3., 4., 1.],
       [1., 1., 1.]])

In [106]:
print(A)
Ut,st,Vt=np.linalg.svd(A, full_matrices=False)
print(Ut)
print(st)
print(Vt)
np.dot(Ut,np.dot(np.diag(st),Vt))

[[1. 2. 3.]
 [2. 1. 4.]
 [3. 4. 1.]
 [1. 1. 1.]]
[[-0.49051953 -0.25988846 -0.83177435]
 [-0.56694707 -0.60603111  0.52369865]
 [-0.61790503  0.75122634  0.12967391]
 [-0.23697042  0.02903905  0.13067451]]
[7.28198365 3.18932577 0.89493871]
[[-0.51017681 -0.58453532 -0.63090259]
 [ 0.25421279  0.59828833 -0.75988613]
 [ 0.82164194 -0.54805979 -0.15663648]]


array([[1., 2., 3.],
       [2., 1., 4.],
       [3., 4., 1.],
       [1., 1., 1.]])